In [32]:
#备注：与preprocessed_NSAP相比，多了一条元路径

In [33]:
#!/usr/bin/env python
# coding: utf-8

import pathlib
import pickle

import numpy as np
import scipy.sparse
import scipy.io
import pandas as pd

In [34]:
num_drug = 1482
num_dis = 793
num_target = 2077
num_gene = 6365

drug_dis=np.load('./raw/txt2npy/us_edges.npy')
#print(drug_dis)
drug_target = np.load('./raw/txt2npy/ut_edges.npy')
gene_dis=np.load('./raw/txt2npy/gs_edges.npy')
print("the number of original drug-dis:",len(drug_dis))
print("the number of original drug-target:",len(drug_target))
print("the number of original gene-dis:",len(gene_dis))

the number of original drug-dis: 11540
the number of original drug-target: 11407
the number of original gene-dis: 18844


In [35]:
drug_dis_table=pd.DataFrame(drug_dis,columns=['drug_id','dis_id'],dtype=np.int32)
#print(drug_dis_table)
drug_target_table=pd.DataFrame(drug_target,columns=['drug_id','target_id'],dtype=np.int32)
#print(drug_target_table)
gene_dis_table=pd.DataFrame(gene_dis,columns=['gene_id','dis_id'],dtype=np.int32)
print(gene_dis_table)

       gene_id  dis_id
0            0     414
1            1      36
2            1      97
3            1     130
4            1     214
...        ...     ...
18839     6362     139
18840     6363     708
18841     6364      42
18842     6364      73
18843     6364     203

[18844 rows x 2 columns]


In [36]:
a=drug_target_table[drug_target_table.drug_id==3]['target_id']
b=np.array(a)
b

array([ 509,  957, 2055,  767,  657,  514,  958,  959,  960], dtype=int32)

In [37]:
b=b[:]+(num_drug+num_dis)
b

array([2784, 3232, 4330, 3042, 2932, 2789, 3233, 3234, 3235], dtype=int32)

In [38]:
drug_dis_table['drug_id'].value_counts().describe()
drug_dis_table['dis_id'].value_counts().describe()
drug_target_table['drug_id'].value_counts().describe()
gene_dis_table['dis_id'].value_counts().describe()

count     793.000000
mean       23.762926
std        75.399571
min         1.000000
25%         1.000000
50%         3.000000
75%        13.000000
max      1074.000000
Name: dis_id, dtype: float64

In [39]:
drug_dis_table['drug_id'].value_counts()  #drug_dis中药物id,及其出现的次数
# drug_dis_table['dis_id'].value_counts() 
#drug_target_table['drug_id'].value_counts()
# drug_target_table['target_id'].value_counts()
# target_gene_table['target_id'].value_counts()
# target_gene_table['gene_id'].value_counts()
# dis_gene_table['gene_id'].value_counts()
# dis_gene_table['dis_id'].value_counts()
# dis_dis_table['dis1_id'].value_counts()
# dis_dis_table['dis2_id'].value_counts()

541     127
397     104
442     100
206      99
631      98
       ... 
1308      1
834       1
826       1
818       1
1467      1
Name: drug_id, Length: 1482, dtype: int64

In [40]:
train_val_test_idx = np.load('./raw/txt2npz/train_val_test_idx.npz')
print(train_val_test_idx.files)
train_idx = train_val_test_idx['train_idx']
val_idx = train_val_test_idx['val_idx']
test_idx = train_val_test_idx['test_idx']
print("drug_dis的数量",len(drug_dis))
print("train_idx的占比，数量，编号:",np.around(len(train_idx)/len(drug_dis),decimals=1),len(train_idx),train_idx)
print("val_idx  的占比，数量，编号:",np.around(len(val_idx)/len(drug_dis),decimals=1),len(val_idx),val_idx)
print("test_idx 的占比，数量，编号:",np.around(len(test_idx)/len(drug_dis),decimals=1),len(test_idx),test_idx)

['train_idx', 'val_idx', 'test_idx']
drug_dis的数量 11540
train_idx的占比，数量，编号: 0.6 6924 [    1     2     3 ... 11534 11535 11536]
val_idx  的占比，数量，编号: 0.2 2308 [    0     7     9 ... 11537 11538 11539]
test_idx 的占比，数量，编号: 0.2 2308 [    5    11    16 ... 11521 11523 11524]


In [41]:
drug_dis_table

,drug_id,dis_id
0,0,14
1,0,84
2,0,716
3,0,726
4,0,278
...,...,...
11535,1481,660
11536,1481,755
11537,1481,758
11538,1481,759


In [42]:
# build the adjacency matrix
# 0 for drug ,1 for disease,2 for target,3 for gene
dim = num_drug + num_dis + num_target + num_gene
print(dim)
type_mask = np.zeros((dim), dtype=int)
type_mask[num_drug:num_drug+num_dis] = 1
type_mask[num_drug+num_dis:dim-num_gene] = 2
type_mask[dim-num_gene:]=3
print(type_mask)

10717
[0 0 0 ... 3 3 3]


In [43]:
adjM = np.zeros((dim, dim), dtype=int)
sum=0
for _, row in drug_dis_table.iterrows():
    uid = row['drug_id']
    sid = num_drug + row['dis_id']
    adjM[uid, sid] += 1
    adjM[sid, uid] += 1
    sum+=1
print("\nthe number of drug-dis:",sum)

sum=0
for _, row in drug_target_table.iterrows():
    uid = row['drug_id']
    tid = num_drug + num_dis + row['target_id']
    adjM[uid, tid] += 1
    adjM[tid, uid] += 1
    sum+=1
print("the number of drug-target:",sum)



sum=0
for _, row in gene_dis_table.iterrows():
    gid = num_drug + num_dis + num_target + row['gene_id']
    did = num_drug + row['dis_id']
    adjM[gid, did] += 1
    adjM[did, gid] += 1
    sum+=1
print("the number of gene-dis:",sum)


the number of drug-dis: 11540
the number of drug-target: 11407
the number of gene-dis: 18844


In [44]:
adjM[1, num_drug:num_drug+num_dis].nonzero()[0]

array([125])

In [45]:
#记录第i行非零元素的索引
drug_dis_list = {i: adjM[i, num_drug:num_drug+num_dis].nonzero()[0] for i in range(num_drug)}
print(drug_dis_list)
dis_drug_list = {i: adjM[num_drug + i, :num_drug].nonzero()[0] for i in range(num_dis)}

drug_target_list={i:adjM[i,num_drug+num_dis:dim-num_gene].nonzero()[0] for i in range(num_drug)}
target_drug_list={i:adjM[num_drug+num_dis+i,:num_drug].nonzero()[0] for i in range(num_target)}
#print(drug_target_list)
dis_dis_list={i:adjM[num_drug+i,num_drug:num_drug+num_dis].nonzero()[0] for i in range(num_dis)}

dis_gene_list={i:adjM[num_drug+i,dim-num_gene:].nonzero()[0] for i in range(num_dis)}
gene_dis_list={i:adjM[dim-num_gene+i,num_drug:num_drug+num_dis].nonzero()[0] for i in range(num_gene)}
#print(gene_dis_list)


{0: array([ 14,  84,  97, 278, 551, 599, 716, 726]), 1: array([125]), 2: array([467]), 3: array([ 60,  61, 227, 425, 613, 734]), 4: array([129, 161, 385]), 5: array([590, 591]), 6: array([  1,   2,  89,  92, 387, 534, 620, 646, 654, 655, 664, 672, 674,
       697, 755, 764, 780, 782]), 7: array([152]), 8: array([ 25,  27,  54,  83,  86, 245, 320, 544, 577, 596, 645, 693, 700,
       716, 750, 753, 757]), 9: array([387]), 10: array([  3,   8,   9,  14,  25,  26,  29,  46,  53,  82, 161, 181, 197,
       198, 217, 230, 247, 256, 272, 312, 393, 535, 537, 538, 539, 540,
       544, 560, 574, 575, 577, 582, 649, 653, 654, 655, 656, 663, 665,
       666, 673, 676, 679, 697, 698, 705, 706, 741, 750, 756, 757, 786,
       792]), 11: array([356, 359]), 12: array([ 25,  27,  54,  83,  86, 245, 320, 544, 577, 596, 645, 693, 700,
       716, 750, 753, 757]), 13: array([ 83, 693, 717]), 14: array([148, 245, 256, 293, 327, 470]), 15: array([ 27, 591, 646, 753]), 16: array([283]), 17: array([ 61, 580

In [46]:
# 0-1-0:drug-dis-drug
u_s_u=[]

for s,u_list in dis_drug_list.items():
    u_s_u.extend((u1,s,u2) for u1 in u_list for u2 in u_list)
u_s_u=np.array(u_s_u)
u_s_u[:,1]+=num_drug
print(u_s_u)
#按[0,2,1]的顺序对元路径进行排序，返回排序后的索引
sorted_index = sorted(list(range(len(u_s_u))), key=lambda i : u_s_u[i, [0, 2, 1]].tolist())
#print(sorted_index)
#按照索引进行排序
u_s_u = u_s_u[sorted_index]
print("\nthe number of metapath usu:",len(u_s_u))
print(u_s_u)

[[  68 1482   68]
 [  68 1482   69]
 [  68 1482   73]
 ...
 [1445 2274 1408]
 [1445 2274 1431]
 [1445 2274 1445]]

the number of metapath usu: 390696
[[   0 1496    0]
 [   0 1566    0]
 [   0 1579    0]
 ...
 [1481 2240 1481]
 [1481 2241 1481]
 [1481 2265 1481]]


In [47]:
#0-2-0:drug-target-drug
u_t_u=[]
for t,u_list in target_drug_list.items():
    u_t_u.extend((u1,t,u2) for u1 in u_list for u2 in u_list)
u_t_u=np.array(u_t_u)
u_t_u[:,1]+=num_drug+num_dis
print(u_t_u)
sorted_index = sorted(list(range(len(u_t_u))), key=lambda i : u_t_u[i, [0, 2, 1]].tolist())
u_t_u = u_t_u[sorted_index]
print("\nthe number of metapath utu:",len(u_t_u))
print(u_t_u)

[[ 196 2275  196]
 [  76 2276   76]
 [  76 2276   85]
 ...
 [ 758 4349  758]
 [1013 4350 1013]
 [1391 4351 1391]]

the number of metapath utu: 1060747
[[   0 2789    0]
 [   0 2834    0]
 [   0 2932    0]
 ...
 [1481 3043 1358]
 [1481 2926 1481]
 [1481 3043 1481]]


In [48]:
#1-3-1:dis-gene-dis
s_g_s=[]
for g,s_list in gene_dis_list.items():
    s_g_s.extend((s1,g,s2) for s1 in s_list for s2 in s_list)  
s_g_s=np.array(s_g_s)
#print(s_g_s)
s_g_s +=num_drug
s_g_s[:,1]+=num_dis+num_target
print(s_g_s)
sorted_index = sorted(list(range(len(s_g_s))), key=lambda i : s_g_s[i, [0, 2, 1]].tolist())
s_g_s = s_g_s[sorted_index]
print("\nthe number of metapath sgs:",len(s_g_s))
print(s_g_s)

[[ 1896  4352  1896]
 [ 1518  4353  1518]
 [ 1518  4353  1579]
 ...
 [ 1685 10716  1524]
 [ 1685 10716  1555]
 [ 1685 10716  1685]]

the number of metapath sgs: 171776
[[1482 6211 1482]
 [1482 6211 1656]
 [1482 6211 1710]
 ...
 [2274 6759 1831]
 [2274 6759 2274]
 [2274 9694 2274]]


In [49]:
#0-1-3-1-0:drug-dis-gene-dis-drug
u_s_g_s_u = []
for s1, g, s2 in s_g_s:
    if len(dis_drug_list[s1 - num_drug]) == 0 or len(dis_drug_list[s2 - num_drug]) == 0:
        continue
    # 取出其中1/5的元素（不能重复）作为u1的候选元素
    candidate_u1_list = np.random.choice(len(dis_drug_list[s1 - num_drug]), int(0.2 * len(dis_drug_list[s1 - num_drug])), replace=False)
    candidate_u1_list = dis_drug_list[s1 - num_drug][candidate_u1_list]
    # 取出其中1/5的元素（不能重复）作为u2的候选元素
    candidate_u2_list = np.random.choice(len(dis_drug_list[s2 - num_drug]), int(0.2 * len(dis_drug_list[s2 - num_drug])), replace=False)
    candidate_u2_list = dis_drug_list[s2 - num_drug][candidate_u2_list]
    u_s_g_s_u.extend([(u1, s1, g, s2, u2) for u1 in candidate_u1_list for u2 in candidate_u2_list])
u_s_g_s_u = np.array(u_s_g_s_u)
sorted_index = sorted(list(range(len(u_s_g_s_u))), key=lambda i : u_s_g_s_u[i, [0, 4, 1, 2, 3]].tolist())
u_s_g_s_u = u_s_g_s_u[sorted_index]
print("\nthe number of metapath usgsu:",len(u_s_g_s_u))
print(u_s_g_s_u)


the number of metapath usgsu: 939012
[[   0 1566 7066 1566    0]
 [   0 1760 5721 1760    0]
 [   0 1566 6570 1543    3]
 ...
 [1481 2265 8510 2086 1083]
 [1481 2265 8510 1567 1210]
 [1481 2265 8510 1567 1216]]


In [50]:
#1-0-1:dis-drug-dis
s_u_s=[]
for u,s_list in drug_dis_list.items():
    s_u_s.extend((s1,u,s2) for s1 in s_list for s2 in s_list)
s_u_s=np.array(s_u_s)
s_u_s[:,[0,2]]+=num_drug
print(s_u_s)
sorted_index = sorted(list(range(len(s_u_s))), key=lambda i : s_u_s[i, [0, 2, 1]].tolist())
s_u_s = s_u_s[sorted_index]
print("\nthe number of metapath sus:",len(s_u_s))
print(s_u_s)

[[1496    0 1496]
 [1496    0 1566]
 [1496    0 1579]
 ...
 [2265 1481 2240]
 [2265 1481 2241]
 [2265 1481 2265]]

the number of metapath sus: 379950
[[1482   68 1482]
 [1482   69 1482]
 [1482   73 1482]
 ...
 [2274 1408 2274]
 [2274 1431 2274]
 [2274 1445 2274]]


In [51]:
u_t_u

array([[   0, 2789,    0],
       [   0, 2834,    0],
       [   0, 2932,    0],
       ...,
       [1481, 3043, 1358],
       [1481, 2926, 1481],
       [1481, 3043, 1481]])

In [52]:
#1-0-2-0-1:dis-drug-target-drug-dis
s_u_t_u_s = []
for u1, t, u2 in u_t_u:
    if len(drug_dis_list[u1]) == 0 or len(drug_dis_list[u2]) == 0:
        continue
    # 取出其中1/5的元素（不能重复）作为s1的候选元素
    candidate_s1_list = np.random.choice(len(drug_dis_list[u1]), int(0.2 * len(drug_dis_list[u1])), replace=False)
    candidate_s1_list = drug_dis_list[u1][candidate_s1_list]
    # 取出其中1/5的元素（不能重复）作为s2的候选元素
    candidate_s2_list = np.random.choice(len(drug_dis_list[u2]), int(0.2 * len(drug_dis_list[u2])), replace=False)
    candidate_s2_list = drug_dis_list[u2][candidate_s2_list]
    s_u_t_u_s.extend([(s1, u1, t, u2, s2) for s1 in candidate_s1_list for s2 in candidate_s2_list])
s_u_t_u_s = np.array(s_u_t_u_s)
s_u_t_u_s[:,[0,4]]+=num_drug
sorted_index = sorted(list(range(len(s_u_t_u_s))), key=lambda i : s_u_t_u_s[i, [0, 4, 1, 2, 3]].tolist())
s_u_t_u_s = s_u_t_u_s[sorted_index]
print("\nthe number of metapath sutus:",len(s_u_t_u_s))
print(s_u_t_u_s)


the number of metapath sutus: 4143032
[[1482   92 3693  397 1482]
 [1482   96 2330   96 1482]
 [1482  282 3438  797 1482]
 ...
 [2274 1445 3438  224 2274]
 [2274 1445 3694  794 2274]
 [2274 1445 3694 1035 2274]]


In [53]:
#（1，0，2，0,1）为新增的元路径
expected_metapaths = [
    [(0, 1, 0), (0,2,0),(0,1,3,1,0)],
    [(1, 0, 1), (1,0,2,0,1),(1, 3, 1)]
]

In [54]:
metapath_indices_mapping = {(0, 1, 0):u_s_u,
                            (0,2,0):u_t_u,
                            (0,1,3,1,0):u_s_g_s_u,
                            (1, 0, 1):s_u_s,
                            (1,0,2,0,1):s_u_t_u_s, 
                            (1, 3, 1):s_g_s                         
                            }

In [55]:
# create the directories if they do not exist
# save_prefix = './preprocess_NSAP/'
save_prefix = './BackupFile/'
for i in range(len(expected_metapaths)):
    pathlib.Path(save_prefix + '{}'.format(i)).mkdir(parents=True, exist_ok=True)

In [56]:
# write all things
target_idx_lists = [np.arange(num_drug), np.arange(num_dis)]
#print(target_idx_lists)
offset_list = [0, num_drug]

In [57]:
for i, metapaths in enumerate(expected_metapaths):
    for metapath in metapaths:
        edge_metapath_idx_array = metapath_indices_mapping[metapath]

        with open(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.pickle', 'wb') as out_file:
            target_metapaths_mapping = {}
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                #若元路径中的第一个结点与当前目标结点相同，则扩展窗口
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                #将窗口内的元路径数据存入目标结点中
                target_metapaths_mapping[target_idx] = edge_metapath_idx_array[left:right, ::-1]
                #滑动窗口
                left = right
            #封装数据
            pickle.dump(target_metapaths_mapping, out_file)

        #np.save(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.npy', edge_metapath_idx_array)

        with open(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '.adjlist', 'w') as out_file:
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                #若元路径中的第一个结点与当前目标结点相同，则扩展窗口
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                #目标结点基于给定元路径metapath的邻居
                neighbors = edge_metapath_idx_array[left:right, -1] - offset_list[i]
                neighbors = list(map(str, neighbors))
                if len(neighbors) > 0:
                    out_file.write('{} '.format(target_idx) + ' '.join(neighbors) + '\n')
                else:
                    out_file.write('{}\n'.format(target_idx))
                left = right


In [58]:
scipy.sparse.save_npz(save_prefix + 'adjM.npz', scipy.sparse.csr_matrix(adjM,dtype=np.int32))
np.save(save_prefix + 'node_types_NSAP.npy', type_mask)

In [59]:
# output positive and negative samples for training, validation and testing

np.random.seed(2022)
# save_prefix = './preprocess_NSAP/'
num_drug = 1482
num_dis = 793
drug_dis = np.load('./raw/txt2npy/us_edges.npy')
train_val_test_idx = np.load('./raw/txt2npz/train_val_test_idx.npz')
train_idx = train_val_test_idx['train_idx']
val_idx = train_val_test_idx['val_idx']
test_idx = train_val_test_idx['test_idx']

neg_candidates = []
counter = 0
for i in range(num_drug):
    for j in range(num_dis):
        if counter < len(drug_dis):
            if i == drug_dis[counter, 0] and j == drug_dis[counter, 1]:
                counter += 1
            else:
                neg_candidates.append([i, j])
        else:
            neg_candidates.append([i, j])
neg_candidates = np.array(neg_candidates)
print("neg_candidates.dtype:",neg_candidates.dtype)


neg_candidates.dtype: int64


In [60]:
#原来的负样本生成
idx = np.random.choice(len(neg_candidates), len(val_idx) + len(test_idx), replace=False)
val_neg_candidates = neg_candidates[sorted(idx[:len(val_idx)])]
test_neg_candidates = neg_candidates[sorted(idx[len(val_idx):])]
print(idx)
print(len(idx))

[987211    180 883432 ... 941403 731749 116994]
4616


In [61]:
train_drug_dis = drug_dis[train_idx]
train_neg_candidates = []
counter = 0
for i in range(num_drug):
    for j in range(num_dis):
        if counter < len(train_drug_dis):
            if i == train_drug_dis[counter, 0] and j == train_drug_dis[counter, 1]:
                counter += 1
            else:
                train_neg_candidates.append([i, j])
        else:
            train_neg_candidates.append([i, j])
train_neg_candidates = np.array(train_neg_candidates)
print("neg sample have completed.")

neg sample have completed.


In [62]:
np.savez(save_prefix + 'train_val_test_neg_drug_dis.npz',
         train_neg_drug_dis=train_neg_candidates,
         val_neg_drug_dis=val_neg_candidates,
         test_neg_drug_dis=test_neg_candidates)
np.savez(save_prefix + 'train_val_test_pos_drug_dis.npz',
         train_pos_drug_dis=drug_dis[train_idx],
         val_pos_drug_dis=drug_dis[val_idx],
         test_pos_drug_dis=drug_dis[test_idx])